# GM - General Motors Company

In [1]:
import yfinance as yf
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ticker = 'GM'
stock = yf.Ticker(ticker)
yearly_balance_sheet = stock.balance_sheet
quarterly_balance_sheet = stock.quarterly_balance_sheet
yearly_cash_flow = stock.cash_flow
quarterly_cash_flow = stock.quarterly_cash_flow
yearly_income_statement = stock.income_stmt
quarterly_income_statement = stock.quarterly_income_stmt
price = stock.info['currentPrice']
volume = stock.info['volume']

In [3]:
print(f"Name: {stock.info['shortName']}")
print(f"Sector: {stock.info['sector']}")
print(f"Industry: {stock.info['industry']}")
print("")
print(f"Price: ${price}")
print(f"Volume: {'{:,}'.format(volume)}")
print("")
print(f"Next Earnings: {stock.calendar['Earnings Date'][0]}")

Name: General Motors Company
Sector: Consumer Cyclical
Industry: Auto Manufacturers

Price: $54.22
Volume: 6,129,189

Next Earnings: 2025-01-28


### About the Company

In [4]:
stock.info['longBusinessSummary']

'General Motors Company designs, builds, and sells trucks, crossovers, cars, and automobile parts; and provide software-enabled services and subscriptions worldwide. The company operates through GM North America, GM International, Cruise, and GM Financial segments. It markets its vehicles primarily under the Buick, Cadillac, Chevrolet, GMC, Baojun, and Wuling brand names. In addition, the company sells trucks, crossovers, cars, and automobile parts through retail dealers, and distributors and dealers, as well as to fleet customers, including daily rental car companies, commercial fleet customers, leasing companies, and governments. Further, it offers range of after-sale services through dealer network, such as maintenance, light repairs, collision repairs, vehicle accessories, and extended service warranties. Additionally, the company provides automotive financing; and software-enabled services and subscriptions. General Motors Company was founded in 1908 and is headquartered in Detroi

##### Company Age

In [5]:
year_founded = 1908
company_age = datetime.datetime.now().year - year_founded
print(f"{company_age} Years")

117 Years


##### CEO

In [6]:
stock.info['companyOfficers'][0]["name"]

'Ms. Mary T. Barra'

##### Full-Time Employees

In [7]:
"{:,}".format(stock.info['fullTimeEmployees'])

'163,000'

##### Company Culture

Rec to a Friend: 64%  
CEO Approval: 58%

### Insider Activity

##### Percent Owned by Insiders

In [8]:
"{:.2f}%".format(stock.info.get('heldPercentInsiders') * 100)

'0.22%'

This is a low percentage, but this is also an old company, so that is okay here.

##### Last 6 Months Transactions

In [9]:
last_6m_insiders = {
    'Purchases': {'Shares': 929699, 'Transactions': 9},
    'Sales': {'Shares': 1252396, 'Transactions':7}
}

share_purch_to_sales = last_6m_insiders['Purchases']['Shares'] / last_6m_insiders['Sales']['Shares']
print(f"Share Purch/Sales: {'{:.2f}'.format(share_purch_to_sales)}")

Share Purch/Sales: 0.74


### Market Cap and Enterprise Value

In [10]:
market_cap = stock.info['marketCap']
print(f"Market Cap: ${'{:.2f}'.format(market_cap / 1000000000)}B")

Market Cap: $59.62B


This is a Large Cap stock.

In [11]:
ev = stock.info['enterpriseValue']
print(f"Enterprise Value: ${'{:.2f}'.format(ev / 1000000000)}B")

Enterprise Value: $165.00B


**<span style="color:red">EV > MC</span>**  

This indicates there is more debt than cash. Where did the debt come from? Is it something that would lead to future growth?

### Financial Indicators

##### P/E 

In [12]:
print(f"Trailing P/E: {stock.info['trailingEps']}")
print(f"Forward P/E: {stock.info['forwardEps']}")

Trailing P/E: 9.37
Forward P/E: 10.57


##### Beta  

<span style="color:red">Risky</span>

In [13]:
stock.info.get("beta", None)

1.414

##### Book Value

In [14]:
# Find Total Current Assets
total_assets = None
possible_keys = [
    'Total Current Assets',
    'Total Assets'
]
for key in possible_keys:
    if key in yearly_balance_sheet.index:
        total_assets = yearly_balance_sheet.loc[key]
        total_assets = total_assets.iloc[0] # most recent
        break

# Find Total Current Liabilities
total_liabilities = None
possible_keys = [
    'Total Current Liabilities',
    'Total Liabilities',
    'Current Liabilities'
]
for key in possible_keys:
    if key in yearly_balance_sheet.index:
        total_liabilities = yearly_balance_sheet.loc[key]
        total_liabilities = total_liabilities.iloc[0] # most recent
        break

book_value = total_assets - total_liabilities
market_to_book = market_cap / book_value

print(f"Book Value: {book_value / 1000000000}")
print(f"Market/Book: {'{:.2f}'.format(market_to_book)}")

Book Value: 178.619
Market/Book: 0.33


##### D/E (Debt to Equity)

In [15]:
# Total Shareholder Equity
shareholder_equity = None
possible_keys = [
    'Total Shareholder Equity',
    'Stockholders Equity'
]

for key in possible_keys:
    if key in yearly_balance_sheet.index:
        shareholder_equity = quarterly_balance_sheet.loc[key]
        shareholder_equity = shareholder_equity.iloc[0]
        break

# Debt
# short_term_debt
short_term_debt = None
possible_keys = [
    'Total Current Liabilities',
    'Total Liabilities',
    'Current Liabilities'
]
for key in possible_keys:
    if key in quarterly_balance_sheet.index:
        short_term_debt = yearly_balance_sheet.loc[key]
        short_term_debt = short_term_debt.iloc[0] # most recent
        break

# Long term debt
long_term_debt = quarterly_balance_sheet.loc['Total Non Current Liabilities Net Minority Interest']
long_term_debt = long_term_debt.iloc[0]

# total debt
debt = short_term_debt + long_term_debt

# Debt to Equity
if shareholder_equity:
    print(f"{'{:.2f}'.format(debt / shareholder_equity)}")
else:
    print(f"{'{:.2f}'.format(debt / book_value)}")

3.02


##### D/C (Debt to Capital)

In [16]:
# Debt to Capital
if shareholder_equity:
    print(f"{'{:.2f}'.format(debt / (debt + shareholder_equity))}")
else:
    print(f"{'{:.2f}'.format(debt / (debt + book_value))}")

0.75


##### ROE  

Only $0.04 made for every dollar invested.

In [17]:
# Total Shareholder Equity
shareholder_equity = None
possible_keys = [
    'Total Shareholder Equity',
    'Stockholders Equity',
    'Total Equity'
]

for key in possible_keys:
    if key in yearly_balance_sheet.index:
        shareholder_equity = quarterly_balance_sheet.loc[key]
        shareholder_equity = shareholder_equity.iloc[0]
        break

# Net Income
net_income = quarterly_income_statement.loc['Net Income'].iloc[0]

# Return on Equity
if shareholder_equity:
    print(f"{'{:.2f}'.format(net_income / shareholder_equity * 100)}%")
else:
    print(f"{'{:.2f}'.format(net_income / book_value * 100)}%")

4.31%


##### Current Ratio

In [18]:
"{:.2f}".format(total_assets / total_liabilities)

'2.89'

##### Inventory Turnover Ratio  

Inventory was turned over about 9 times per year.

In [19]:
# Cost of Good Sold
cogs = yearly_income_statement.loc['Cost Of Revenue']
cogs = cogs.iloc[0]

# Inventory
inventory = yearly_balance_sheet.loc['Inventory']
inventory = inventory.iloc[0]

# Inventory Turnover Ratio
"{:.2f}".format(cogs / inventory)

'9.28'

##### DSI (Days Sales of Inventory)  

It takes about 40 days to sale through inventory.

In [20]:
"{:.2f}".format(inventory / cogs * 365)

'39.35'

### Balance Sheet

##### Quarterly Cash and Long-Term Debt

##### Cash and Cash Equivalents  

Cash has been **<span style="color:red">decreasing</span>** YoY. 

In [ ]:
# Series for cash and equivs where index is dates and cash is shown in billions
quarterly_cash_and_equivs = quarterly_balance_sheet.loc['Cash And Cash Equivalents'] / 1000000000
yearly_cash_and_equivs = yearly_balance_sheet.loc['Cash And Cash Equivalents'] / 1000000000

# Create the subplots (1 row, 2 columns)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot Yearly Cash on the first subplot
axes[0].plot(
    yearly_cash_and_equivs.index, 
    yearly_cash_and_equivs.values, 
    marker='o', 
    color='#3cb1fa', 
    linestyle='-', 
    label='Series 1'
)
axes[0].set_title('Yearly Cash')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('$ (Billions)')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)  # Rotate x-axis labels

# Plot Quarterly Cash on the second subplot
axes[1].plot(
    quarterly_cash_and_equivs.index, 
    quarterly_cash_and_equivs.values, 
    marker='o', 
    color='#e43cfa', 
    linestyle='-', 
    label='Series 2'
)
axes[1].set_title('Quarterly Cash')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('$ (Billions)')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)  # Rotate x-axis labels

# Adjust layout to avoid overlap
plt.tight_layout()

# Show the plot
plt.show()

##### Long-Term Debt  

Long-term debt has been **increasing** YoY. Have there been big investments?

In [ ]:
# Series for long term debt where index is dates and debt is shown in Billions
yearly_long_term_debt = yearly_balance_sheet.loc['Long Term Debt'] / 1000000000
quarterly_long_term_debt = quarterly_balance_sheet.loc['Long Term Debt'] / 1000000000

# Create the subplots (1 row, 2 columns)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot Yearly Debt on the first subplot
axes[0].plot(
    yearly_long_term_debt.index, 
    yearly_long_term_debt.values, 
    marker='o', 
    color='#3cb1fa', 
    linestyle='-', 
    label='Series 1'
)
axes[0].set_title('Yearly Debt')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('$ (Billions)')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)  # Rotate x-axis labels

# Plot Quarterly Debt on the second subplot
axes[1].plot(
    quarterly_long_term_debt.index, 
    quarterly_long_term_debt.values, 
    marker='o', 
    color='#e43cfa', 
    linestyle='-', 
    label='Series 2'
)
axes[1].set_title('Quarterly Debt')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('$ (Billions)')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)  # Rotate x-axis labels

# Adjust layout to avoid overlap
plt.tight_layout()

# Show the plot
plt.show()

##### Total Shares Outstanding

### Cash Flow

In [ ]:
# Net Cash Flow by year
yearly_net_cf = None
possible_keys = [
    'Net Cash Flow',
    'Total Cash From Operating Activities',
    'Cash Flow From Continuing Operating Activities'
]
for key in possible_keys:
    if key in yearly_cash_flow.index:
        yearly_net_cf = yearly_cash_flow.loc[key] / 1000000000
        break

# Net Cash Flow by quarter
quarterly_net_cf = None
possible_keys = [
    'Net Cash Flow',
    'Total Cash From Operating Activities',
    'Cash Flow From Continuing Operating Activities'
]
for key in possible_keys:
    if key in quarterly_cash_flow.index:
        quarterly_net_cf = quarterly_cash_flow.loc[key] / 1000000000
        break

# Net Cash Flow by year
yearly_operating_cf = yearly_cash_flow.loc['Operating Cash Flow'] / 1000000000
quarterly_operating_cf = quarterly_cash_flow.loc['Operating Cash Flow'] / 1000000000

yearly_investing_cf = yearly_cash_flow.loc['Investing Cash Flow'] / 1000000000
quarterly_investing_cf = quarterly_cash_flow.loc['Investing Cash Flow'] / 1000000000

In [ ]:
# Yearly net, operating, and investing cash flow
plt.figure(figsize=(10, 6))
plt.plot(
    yearly_net_cf.index, 
    yearly_net_cf.values, 
    marker='o', 
    color='#ab3cfa', 
    linestyle='-', 
    label='Net CF'
)
plt.plot(
    yearly_operating_cf.index, 
    yearly_operating_cf.values, 
    marker='o', 
    color='#fa3cb1', 
    linestyle='--', 
    label='Operating CF'
)
plt.plot(
    yearly_investing_cf.index,
    yearly_investing_cf.values,
    marker='o',
    color='#3cfa85',
    linestyle='--',
    label='Investing CF'
)
plt.title('Yearly Cash Flow')
plt.xlabel('Date')
plt.ylabel('Cash Flow (Billion $)')
plt.legend()
plt.xticks(rotation=45)  # Rotate x-axis labels
plt.tight_layout()

# Quarterly Net, Operating, and Investing Cash Flow
plt.figure(figsize=(10, 6))
plt.plot(
    quarterly_net_cf.index, 
    quarterly_net_cf.values, 
    marker='s', 
    color='#ab3cfa', 
    linestyle='-', 
    label='Net CF'
)
plt.plot(
    quarterly_operating_cf.index, 
    quarterly_operating_cf.values, 
    marker='^', 
    color='#fa3cb1', 
    linestyle='--', 
    label='Operating CF'
)
plt.plot(
    quarterly_investing_cf.index, 
    quarterly_investing_cf.values, 
    marker='^', 
    color='#3cfa85', 
    linestyle='--', 
    label='Investing CF'
)
plt.title('Quarterly Cash Flow')
plt.xlabel('Date')
plt.ylabel('Value (Billions)')
plt.legend()
plt.xticks(rotation=45)  # Rotate x-axis labels
plt.tight_layout()

plt.show()